In [1]:
import uproot
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, BaseSchema, NanoAODSchema
import numpy as np
import ROOT as r

Welcome to JupyROOT 6.24/07


In [2]:
fileToSkim   = "root://cms-xrd-global.cern.ch///store/mc/RunIISummer19UL17NanoAODv2/QCD_HT1000to1500_TuneCP5_PSWeights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/250000/032CD0E4-6F4F-A049-A65C-0A24275AB0C2.root"
outputFile   = "test.root"

In [4]:
#Taken from https://github.com/CoffeaTeam/coffea/discussions/735

def is_rootcompat(a):
    """Is it a flat or 1-d jagged array?"""
    t = ak.type(a)
    if isinstance(t, ak._ext.ArrayType):
        if isinstance(t.type, ak._ext.PrimitiveType):
            return True
        if isinstance(t.type, ak._ext.ListType) and isinstance(t.type.type, ak._ext.PrimitiveType):
            return True
    return False

def uproot_writeable(events):
    """Restrict to columns that uproot can write compactly"""
    out = {}
    for bname in events.fields:
        if events[bname].fields:
            out[bname] = ak.zip(
                {
                    n: ak.packed(ak.without_parameters(events[bname][n]))
                    for n in events[bname].fields
                    if is_rootcompat(events[bname][n])
                }
            )
        else:
            out[bname] = ak.packed(ak.without_parameters(events[bname]))
    return out

In [5]:
eventsToRead = None
#eventsToRead = 10000
events = NanoEventsFactory.from_root(fileToSkim,schemaclass=NanoAODSchema,entry_stop=eventsToRead).events()

/users/mrogul/Work/H3/H3env/lib/python3.9/site-packages/uproot/reading.py:171: FutureWarning: XRootD unknown is not fully supported; either upgrade to 5.2.0+ or set

    open.defaults["xrootd_handler"] = uproot.MultithreadedXRootDSource

  warnings.warn(message, FutureWarning)


In [6]:
ptCut  = 200
msdCut = 50
etaCut = 2.5

ptMask       = events.FatJet.pt>ptCut
msdMask      = events.FatJet.msoftdrop>msdCut
etaMask      = np.abs(events.FatJet.eta)<etaCut

#We require at least two FatJets satisfying pt, eta and msd requirements
#This allows for 3+0 or 2+1 (AK8Jet+AK4Jet topologies)
skimmingMask = ak.sum(ptMask & msdMask & etaMask, axis=1)>1
skimmed      = events[skimmingMask]

In [7]:
with uproot.recreate(outputFile) as fout:
    fout["Events"] = uproot_writeable(skimmed)
    
#Save Runs tree
#Copying TTrees with uproot is not yet supported so we use ROOT directly
inputFile  = r.TFile.Open(fileToSkim)
runsTree   = inputFile.Get("Runs")
outputFile = r.TFile.Open(outputFile,"UPDATE")
outputFile.cd()
runsTree.CloneTree().Write()
outputFile.Close()
inputFile.Close()

Warning in <TClass::Init>: no dictionary for class __pair_base<edm::Hash<1>,edm::ParameterSetBlob> is available
